# Section 4: Feature Extraction

Sometimes the attributes extracted cannot be used directly in a machine learning model. From the attributes presented in previous section, for example, only the numeric attributes (which values are real or integer) could be used directly in a model. The list of words extracted, such as libraries and system calls used by a software, must pass through one more step called feature extraction, whose objective is to transform these attributes into something readable by the classifier, usually defining the same number of features as output of this process (once a program can use more libraries than others, for example). In this section we introduce how to extract features from textual attributes (using TF-IDF, Word2Vec and Encoders) and how to normalize them. 

## Methods

We are going to present two textual feature extraction methods, both of them used a lot in the literature: TF-IDF and Word2Vec. Given a text representing an attribute, both methods produce as output a numeric list representing in a different way this text. It is important to note that the list of words extracted in previous section are transformed into texts, where each word is separated by a space. 

### TF-DF

Given a vocabulary from a set of texts, i.e., all the words that appear in these texts, each text ![i](imgs/tfidf1.png) is represented by a vector ![vec](imgs/tfidf2.png), where ![wij](imgs/tfidf3.png) represents the TF-IDF (Term Frequency - Inverse Document Frequency) from the j-th word of the vocabulary. The TF-IDF is as statistic measure used to evaluate how important a word is to a text in relation to a collection of texts [Manning et al. 2008a]. This measure is obtained through the multiplication of two terms:

* **Term Frequency (TF):** measure how often a word ![t](imgs/tfidf4.png) occurs in a text through the equation ![tf](imgs/tfidf5.png).

* **Inverse Document Frequency (IDF):** measure how important a word ![t](imgs/tfidf4.png) is through the equation ![idf](imgs/tfidf6.png).

Briefly, each text is represented by a sparse array that contains their TF-IDF values for each word in the vocabulary. The vocabulary can be reduced to a number V of words, the most prevalent in the set of texts.

Using the Brazilian Malware dataset, below we present a way to extract features from the list of libraries used by each sample using TF-IDF, through the class [*TfidfVectorizer*](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) from scikit-learn [Pedregosa et al. 2011]. First, the dataset is read, as below.

In [1]:
import pandas as pd
# dataset location
data_path = "./datasets/brazilian-malware.csv"
# read CSV dataset
data = pd.read_csv(data_path)

Then, the extractor is initialized with a maximum of 200 features (each feature represents a word) and the set of samples is divided into two: the first set is used to train the extractor (using the method *fit*, which generates the vocabulary used) and the later, to test. Finally, both sets have their features extracted through the method *transform*. Looking at the output, we can see that the resulting matrix has 200 columns, the maximum number of features we selected.

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
# get texts
texts = data["ImportedDlls"].values
# get mid of data
mid = int((len(texts) + 1)/2)
# split data into train and test
train_texts = texts[:mid]
test_texts = texts[mid:]
# initialize vectorizer
vectorizer = TfidfVectorizer(max_features=200)
# train vectorizer
vectorizer.fit(train_texts)
# transform train texts to features
train_features = vectorizer.transform(train_texts)
test_features = vectorizer.transform(test_texts)
# print features shape
print(train_features.shape, test_features.shape)

(25091, 200) (25090, 200)


### Word2Vec

Developed in 2013 by Google researchers [Mikolov et al. 2013a], word2vec is a way to create vectors that represent words. The greatest contribution of this method is its capability of representing the similarity between words and their meanings, different from TF-IDF, which just project data based on their frequency. Besides that, word2vec projects only words and not the text, making necessary to use a strategy to extract features from the projected words. The word2vec general model uses a feed-forward non-supervised neural network (for more details about neural networks, check [section 5](05_models.ipynb)). There are two architectures available for this model: continuous bag-of-words (CBOW) e skip-gram. The first one (CBOW) tries to predict the central word from a set of words around them (input), similar to a cluster. In contrast, the skip-gram architecture does the opposite: it tries to predict the words around, given a central word as input. The figure below presents both methods [Shulman 2016, Mikolov et al. 2013b]. CBOW was the architecture used in this work and will be explained in details further.

<img align="center" src="imgs/word2vec1.png">

First, each input of the network is codified into an vector ![x_ck](imgs/word2vec2.png), where V is the vocabulary size, *c* is the word identifier and *k*, the text identifier (as presented in the figure below). The item ![x_i](imgs/word2vec3.png) is 1 only if *i* corresponds to the word identifier *c*. Otherwise, ![x_i](imgs/word2vec3.png) is 0, i.e., only one position of the array of each word will have value 1 (only the one that corresponds to the word). This codification is called one-hot encoding. Through this input, the model calculates the average of these arrays and multiplies it by the weights from the hidden layer *h*, generating the output *y*. More details about the training of this network can be found in [Rong 2014].

<img align="center" src="imgs/word2vec4.png">

The code bellow presents a way to extract features using word2vec. For that, we implemented a class called *MeanEmbeddingVectorizer*, which uses the model [*Word2Vec*](https://radimrehurek.com/gensim/models/word2vec.html) implemented by the library gensim [Rehurek and Sojka 2010]. In this class, we defined only two parameters (more parameters can be added when modifying this class: *size*, which defines the dimension used to project the words and *min_count*, which defines the minimum of apparition for a word to be considered. In the method *fit* we train the *Word2Vec* model with the train data and obtain the projection of these data. With the method *transform*, the data passed as parameter use the created projection to create a feature vector for each sample, based on the average of their words (in case the word does not exist, it is considered as being a null vector). Thus, the class that we implemented works very similar to the one implemented by TF-IDF's scikit-learn.

In [27]:
from gensim.models import Word2Vec
# vectorizer class: calc average of words using word2vec
class MeanEmbeddingVectorizer(object):
    def __init__(self, size, min_count=1): #word2vec):
        self.size = size
        self.min_count = 1

    def fit(self, X):
        w2v = Word2Vec(X,size=self.size,min_count=self.min_count)
        self.word2vec = dict(zip(w2v.wv.index2word,w2v.wv.vectors))
        # if a text is empty we should return a vector of zeros
        # with the same dimensionality as all the other vectors
        self.dim = len(list(self.word2vec.values())[0])
        return self

    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec]
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])
# get texts
texts = data["ImportedDlls"].values
# get mid of data
mid = int((len(texts) + 1)/2)
# split data into train and test
train_texts = texts[:mid]
test_texts = texts[mid:]
# initialize vectorizer
m = MeanEmbeddingVectorizer(size=200)
# train vectorizer
m.fit(train_texts)
# transform train and test texts to w2v mean
train_features = m.transform(train_texts)
test_features = m.transform(test_texts)
# print features shape
print(train_features.shape, test_features.shape)

(25091, 200) (25090, 200)


## Normalization

TODO

### Min-Max

TODO

In [32]:
from sklearn.preprocessing import MinMaxScaler
# initialize minmax
scaler = MinMaxScaler(feature_range=(0, 1))
# train minmax
scaler.fit(train_features.toarray())
# transform features
train_features_norm = scaler.transform(train_features.toarray())
test_features_norm = scaler.transform(test_features.toarray())
print(train_features_norm.shape, test_features_norm.shape)

(25091, 200) (25090, 200)


### Standardization

TODO

In [ ]:
from sklearn.preprocessing import StandardScaler
# initialize minmax
scaler = StandardScaler()
# train minmax
scaler.fit(train_features.toarray())
# transform features
train_features_norm = scaler.transform(train_features.toarray())
test_features_norm = scaler.transform(test_features.toarray())
print(train_features_norm.shape, test_features_norm.shape)

## References

[Manning et al. 2008a] Manning, C. D., Raghavan, P., and Schütze, H. (2008a). Introduction to Information Retrieval. Cambridge University Press, New York, NY, USA.

[Mikolov et al. 2013a] Mikolov, T., Chen, K., Corrado, G., and Dean, J. (2013a). Efficient estimation of word representations in vector space. CoRR, abs/1301.3781.

[Pedregosa et al. 2011] Pedregosa, F., Varoquaux, G., Gramfort, A., Michel, V., Thirion, B., Grisel, O., Blondel, M., Prettenhofer, P., Weiss, R., Dubourg, V., Vanderplas, J., Passos, A., Cournapeau, D., Brucher, M., Perrot, M., and Duchesnay, E. (2011). Scikitlearn: Machine learning in Python. Journal of Machine Learning Research, 12:2825–2830.

[Rehurek and Sojka 2010] Reh ˚u ˇ ˇrek, R. and Sojka, P. (2010). Software Framework for Topic Modelling with Large Corpora. In Proceedings of the LREC 2010 Workshop on New Challenges for NLP Frameworks, pages 45–50, Valletta, Malta. ELRA. http: //is.muni.cz/publication/884893/en.

[Rong 2014] Rong, X. (2014). word2vec parameter learning explained. CoRR, abs/1411.2738.